# 仅实现item1∧item2->item3的形式

In [18]:
import itertools
import pandas as pd
import numpy as np
data = [[1, 3, 4],
        [3, 2, 5],
        [1, 2, 3, 5],
        [2, 5]]

class Apriori:
    def __init__(self,numer_data,min_support,min_confidence):
        self.numer_data = numer_data
        self.max_len = len(self.numer_data)
        self.unique_item = [1,2,3,4,5,6]
        self.min_support = min_support
        self.min_confidence = min_confidence
    
    #计算单个的支持度    
    def cal_support(self):
        support = []
        for t1 in self.unique_item:
            i = 0 #初始化是否在数组中的计数器
            for t2 in data:
                if t1 in t2:
                    i += 1
            support.append(i/self.max_len)
        return support
    
    #删选支持度过低的
    def del_min_support(self):
        new_data1 = []
        for t1,t2 in zip(self.cal_support(),self.unique_item):
            if t1 >= 0.6:
                new_data1.append(t2)
        return new_data1
    
    #将上一步删选得到的符合支持度条件的单个商品进行两两组合
    def generate_samples(self):
        return itertools.combinations(self.del_min_support(),2)
    
    #由两两组合的商品，计算其支持度，置信度，提升度
    def cal_py(self):
        support2 = []
        support2_samples = []
        i = 0 #初始化两两组合的计数器
        for t1 in self.generate_samples():
            support2_samples.append(t1)
            i += 1 
            j = 0 #初始化支持度计数器
            for t2 in self.numer_data:
                if t1[0] in t2 and t1[1] in t2:
                    j += 1
            support2.append([j])
        # return support2
        return [support2_samples,list((np.array(support2)/self.max_len).reshape(-1))]
    
    #计算support2_samples的支持度
    def cal_conf2(self):
        support2_samples,support2 = self.cal_py()[0],self.cal_py()[1]
        p_y = []
        p_xy = []
        for t1 in support2_samples:
            # t = self.del_min_support()
            y = set(t1)
            i,j = 0,0 #初始化p_y的计数器和p_xy的计数器
            for t2 in self.numer_data:
                if y.issubset(set(t2)):
                    i += 1
                if set(self.del_min_support()).issubset(set(t2)):
                    j += 1
            p_y.append(i/self.max_len)
            p_xy.append(j/self.max_len)
        # print(p_xy)
        return [support2_samples,list((np.array(p_xy).reshape(-1))/(np.array(p_y).reshape(-1)))]
    
     #删掉置信度不达到设定标准的
    def del_min_conf2(self):
        new_data2 = [[],[]]
        for t1,t2 in zip(self.cal_conf2()[0],self.cal_conf2()[1]):
            if t2 >= self.min_confidence:
                new_data2[0].append(t1)
                new_data2[1].append(t2)
        return new_data2
    
    #计算support2_samples的置信度
    def cal_lift2(self):
        support2_samples,support2 = self.del_min_conf2()[0],self.del_min_conf2()[1]
        p_x = []
        for t1 in support2_samples:
            t = self.del_min_support()
            y = set(t)-set(t1)
            # print('y:',y)
            i = 0 #初始化p_y的计数器
            for t2 in self.numer_data:
                # print('t2: ',t2)
                if y.issubset(set(t2)):
                    i += 1
            # print('i:',i)
            p_x.append(i/self.max_len)
        #print(p_x)
        return [support2_samples,np.array(self.del_min_conf2()[1])/np.array(p_x)]

In [19]:
test = Apriori(numer_data=data,min_support=0.5,min_confidence=0.6)
test.cal_py()
# test.cal_conf2()
print('置信度结果: ',test.del_min_conf2())
print('提升度结果: ',test.cal_lift2())

置信度结果:  [[(2, 3), (2, 5), (3, 5)], [1.0, 0.6666666666666666, 1.0]]
提升度结果:  [[(2, 3), (2, 5), (3, 5)], array([1.33333333, 0.88888889, 1.33333333])]


# apriori库的结果

In [21]:
from apyori import apriori

data = [[1, 3, 4],
        [3, 2, 5],
        [1, 2, 3, 5],
        [2, 5]]

result = list(apriori(transactions=data,min_support=0.5,min_confidence=0.6))
len(result)
print(result[-1])


RelationRecord(items=frozenset({2, 3, 5}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset({2}), items_add=frozenset({3, 5}), confidence=0.6666666666666666, lift=1.3333333333333333), OrderedStatistic(items_base=frozenset({3}), items_add=frozenset({2, 5}), confidence=0.6666666666666666, lift=0.8888888888888888), OrderedStatistic(items_base=frozenset({5}), items_add=frozenset({2, 3}), confidence=0.6666666666666666, lift=1.3333333333333333), OrderedStatistic(items_base=frozenset({2, 3}), items_add=frozenset({5}), confidence=1.0, lift=1.3333333333333333), OrderedStatistic(items_base=frozenset({2, 5}), items_add=frozenset({3}), confidence=0.6666666666666666, lift=0.8888888888888888), OrderedStatistic(items_base=frozenset({3, 5}), items_add=frozenset({2}), confidence=1.0, lift=1.3333333333333333)])


#### 比较item1∧item2->item3的部分可以发现，输出结果一致